In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import pandas as pd
from openpyxl import load_workbook
import os

# Path to the folder where files are downloaded
download_folder = r"" # Set your actual download folder path

# Function to monitor the download folder for the latest file
def wait_for_download(download_folder, timeout=60):
    # Get the initial list of files in the folder
    before = set(os.listdir(download_folder))
    
    # Wait for the file to be added
    start_time = time.time()
    while True:
        time.sleep(1)
        after = set(os.listdir(download_folder))
        added_files = after - before
        
        if added_files:
            # Return the first new file (assuming it's the one you just downloaded)
            latest_file = added_files.pop()
            return os.path.join(download_folder, latest_file)
        
        # Timeout logic
        if time.time() - start_time > timeout:
            raise Exception("File download timed out!")

# Initialize Chrome
driver = webdriver.Chrome()

# Open the website
driver.get("https://seller.mm.ru/seller/76728/")

# Wait for the email input field to be visible and input the email
email_xpath = '//input[@type="text" and @id="username"]'
WebDriverWait(driver, 40).until(
    EC.visibility_of_element_located((By.XPATH, email_xpath))
).send_keys("")

# Wait for the password input field to be visible and input the password
password_xpath = '//input[@type="password" and @id="password"]'
WebDriverWait(driver, 40).until(
    EC.visibility_of_element_located((By.XPATH, password_xpath))
).send_keys("")

# Wait for the "Продолжить" button to be clickable and click it
continue_button_xpath = '//button[@data-test-id="button__next"]'
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, continue_button_xpath))
).click()

# Wait for the "Отчеты" button to be clickable and click it
reports_button_xpath = '//em[text()="Отчеты"]'
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, reports_button_xpath))
).click()

# Wait for the "Формирование отчета" button to be clickable and click it
report_creation_button_xpath = '//button[@data-test-id="button__report-creation"]'
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, report_creation_button_xpath))
).click()

# Wait for the input field with placeholder "Выберите магазин" to be visible and click it
store_input_xpath = '//input[@placeholder="Выберите магазин"]'
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, store_input_xpath))
).click()

# Wait for the "Выбрать все" div to be clickable and click it
select_all_xpath = '//div[text()="Выбрать все"]'
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, select_all_xpath))
).click()

# Scroll the element into view and click it
radio_label_xpath = '//label[@for="radio-express_23"]'  # XPath for the desired label element

# Wait for the label to be visible
WebDriverWait(driver, 40).until(
    EC.visibility_of_element_located((By.XPATH, radio_label_xpath))
)

# Scroll the element into view using JavaScript
radio_label = driver.find_element(By.XPATH, radio_label_xpath)
driver.execute_script("arguments[0].scrollIntoView(true);", radio_label)

# Wait briefly for the scroll to complete
time.sleep(1)

# Click the element
radio_label.click()

# XPath for the new label element containing 'XLSX'
radio_label_xpath_xlsx = '//label[@for="radio-express_18"]'  # XPath for the new 'XLSX' label element

# Wait for the label to be clickable
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, radio_label_xpath_xlsx))
)

# Click the element directly
radio_label_xlsx = driver.find_element(By.XPATH, radio_label_xpath_xlsx)
radio_label_xlsx.click()

# XPath for the button using the data-test-id attribute
button_xpath = '//button[@data-test-id="form-report"]'

# Wait for the button to be clickable
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, button_xpath))
)

# Click the button
button = driver.find_element(By.XPATH, button_xpath)
button.click()

time.sleep(5)

# XPath for the button with the close icon (using data-test-id)
close_button_xpath = '//div[@data-test-id="button__close-modal"]'

# Wait for the close button to be clickable
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, close_button_xpath))
)

# Click the close button
close_button = driver.find_element(By.XPATH, close_button_xpath)
close_button.click()

time.sleep(40)

# Wait for the download button to be visible (using a more stable XPath)
download_button_xpath = '//div[contains(@class, "download-icon") and contains(@class, "icon-custom-size")]'

# Wait for the download button to be clickable
WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.XPATH, download_button_xpath))
)

# Click the download button
download_button = driver.find_element(By.XPATH, download_button_xpath)
download_button.click()

########


# Wait for the file to be downloaded and get the file path
downloaded_file_path = wait_for_download(download_folder)

# Read the downloaded file into a pandas DataFrame
df = pd.read_excel(downloaded_file_path)
df.columns = df.iloc[0,:] # make 1st row as column
df = df.iloc[1:,:] # Remove first row since it has now become column header

# SAVING EXTRACTED REPORT TO EXCEL FILE
df.to_excel(r"", index=False)

# SHAREPOINT SITE WHERE YOU WANT TO SAVE YOUR EXCEL FILE
site_url = ''
doc_library = ''
file_name = ''

# Credentialss
username = ''
password = ''


# Local download path
download_path = "./temp.xlsx"

try:
    # Authenticate
    auth_ctx = AuthenticationContext(site_url)
    if not auth_ctx.acquire_token_for_user(username, password):
        print(f"Error acquiring token: {auth_ctx.get_last_error()}")
        exit(1)

    ctx = ClientContext(site_url, auth_ctx)

    # Download the file from SharePoint
    response = File.open_binary(ctx, f"{doc_library}/{file_name}")

    # Check if the response content is not empty
    if not response.content:
        print("No data was downloaded from SharePoint.")
        exit(1)

    # Write the response content to a local file
    with open(download_path, "wb") as local_file:
        local_file.write(response.content)

    print("File has been downloaded from SharePoint.")

except Exception as e:
    print(f"Error during file download: {e}")
    exit(1)


try:
    book = load_workbook(download_path)
    sheet = book['Отчет по продажам']
except Exception as e:
    print(f"Error loading the workbook or sheet: {e}")
    exit(1)


data = sheet.values

# First row as columns
columns = next(data)


existing_df = pd.DataFrame(data, columns=columns)


# columnss
subset_columns = ['Статус',	'Дата создания', 'Дата получения', '№ заказа', 'Штрихкод',	
                  'SKU', 'Наименование',	'Категория', 'Количество', 'Возвраты',	'Выручка (руб.)', 
                  'Выручка с вычетом комиссии (руб.)', 'Комиссия маркетплейса(руб.)', 'Цена (руб.)', 
                  'Промокод (руб.)',	'Себестоимость (руб.)',	'Seller SKU ID']

combined_df = pd.concat([existing_df, df], ignore_index=True).drop_duplicates(subset=subset_columns)


new_rows = combined_df[len(existing_df):]

for row in new_rows.itertuples(index=False, name=None):
    sheet.append(row)

try:
    book.save(download_path)
    print("Data has been successfully written to the Excel file without duplicates.")
except Exception as e:
    print(f"Error saving the Excel file: {e}")
    exit(1)


book.close()

# Uploadingggg the updated file back to SharePoint
try:
    with open(download_path, 'rb') as content_file:
        file_content = content_file.read()

    File.save_binary(ctx, f"{doc_library}/{file_name}", file_content)
    print("File has been uploaded back to SharePoint.")

except Exception as e:
    print(f"Error during file upload: {e}")

# Cleanup: Remove the local temp file
if os.path.exists(download_path):
    os.remove(download_path)
    print("Temporary file has been deleted.")